In [1]:
import pandas as pd
import math
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sys
import gzip as gz
import scipy
from importlib import reload
import glob
import tqdm
import itertools
from scipy.spatial.distance import cdist
import scanpy as sc

import sys
sys.path.append('/Genomics/pritykinlab/dillon/perturbseq/scripts/utils')
import dataloader
import umap_analysis
import reimplementation
import normalization
import adata_utils
from scipy.stats import mannwhitneyu
from scipy import stats

In [2]:
plt.rcParams["axes.titlesize"] = 25
plt.rcParams["axes.labelsize"] = 25
plt.rcParams["xtick.labelsize"] = 20
plt.rcParams["ytick.labelsize"] = 20
plt.rcParams["lines.markersize"] = 5
plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['legend.fontsize'] = 15
plt.rcParams['legend.title_fontsize'] = 15
plt.rcParams['legend.edgecolor'] = 'dimgray'
plt.rcParams['figure.figsize'] = (7,7)

In [3]:
data_dir = "/Genomics/pritykinlab/dillon/other_studies/BassezLambrechts2021"

In [4]:
import os

In [5]:
TCR_df = pd.read_csv(os.path.join(data_dir, "1879-BIOKEY_barcodes_vdj_combined_cohort1.csv.xls"))

In [6]:
metadata_df = pd.read_csv(os.path.join(data_dir, "1870-BIOKEY_metaData_tcells_cohort1_web.csv.xls"))

In [7]:
TCR_df

,barcode,clonotype,cdr3_nt,cdr3_aa,SAMPLE_ID
0,BIOKEY_10_Pre_AAACCTGCATCTCGCT,BIOKEY_10_Pre_clonotype10,TRA:TGTGCAGAGAATATCGGTGGAGGAAGCCAAGGAAATCTCATC...,TRA:CAENIGGGSQGNLIF;TRB:CASSSRSGGIQYF,BIOKEY_10_Pre
1,BIOKEY_10_Pre_AAACCTGGTTCTCATT,BIOKEY_10_Pre_clonotype1,TRA:TGTGCAGCTTCCGACAGAGGCTCAACCCTGGGGAGGCTATAC...,TRA:CAASDRGSTLGRLYF;TRB:CASSLGPGAGQETQYF,BIOKEY_10_Pre
2,BIOKEY_10_Pre_AAACCTGTCAAGAAGT,BIOKEY_10_Pre_clonotype65,TRA:TGTGCTGTGAGTGCGTCGGGTAACACCGACAAGCTCATCTTT...,TRA:CAVSASGNTDKLIF;TRB:CASSTYRGLDMNTEAFF,BIOKEY_10_Pre
3,BIOKEY_10_Pre_AAACCTGTCTCGCATC,BIOKEY_10_Pre_clonotype41,TRA:TGTGCTGTGAGAGACAGCTATACCGGCACTGCCAGTAAACTC...,TRA:CAVRDSYTGTASKLTF;TRB:CASSFRRGDNEQFF,BIOKEY_10_Pre
4,BIOKEY_10_Pre_AAAGATGAGCTCCTTC,BIOKEY_10_Pre_clonotype1,TRA:TGTGCAGCTTCCGACAGAGGCTCAACCCTGGGGAGGCTATAC...,TRA:CAASDRGSTLGRLYF;TRB:CASSLGPGAGQETQYF,BIOKEY_10_Pre
...,...,...,...,...,...
51494,BIOKEY_24_On_TTGTAGGTCCCGGATG,BIOKEY_24_On_clonotype23,TRA:TGTGCTGTGAGTTATTATGGAGGAAGCCAAGGAAATCTCATC...,TRA:CAVSYYGGSQGNLIF;TRB:CASTARASRNYGYTF,BIOKEY_24_On
51495,BIOKEY_24_On_TTTACTGCACCATCCT,BIOKEY_24_On_clonotype308,TRA:TGTGCAGCACGTGACAGCTGGGGGAAATTGCAGTTT,TRA:CAARDSWGKLQF,BIOKEY_24_On
51496,BIOKEY_24_On_TTTACTGTCCATGCTC,BIOKEY_24_On_clonotype309,TRA:TGTGCCGTGAGATACGGGAACAACAGACTCGCTTTT;TRB:T...,TRA:CAVRYGNNRLAF;TRB:CASSRLQGNEQFF,BIOKEY_24_On
51497,BIOKEY_24_On_TTTGCGCCACTGTTAG,BIOKEY_24_On_clonotype310,TRA:TGCATCCTGACTTTTTCGGGGGGGGCTAACTTTGGAAATGAG...,TRA:CILTFSGGANFGNEKLTF;TRB:CASKRSLNTEAFF,BIOKEY_24_On


In [8]:
metadata_df

,Cell,nCount_RNA,nFeature_RNA,patient_id,timepoint,expansion,BC_type,cellSubType,cohort
0,BIOKEY_13_Pre_AAACCTGCAAGAAGAG-1,1252,700,BIOKEY_13,Pre,NaN,HER2+,gdT,treatment_naive
1,BIOKEY_13_Pre_AAACGGGTCGTAGGAG-1,1869,995,BIOKEY_13,Pre,NaN,HER2+,NK_REST,treatment_naive
2,BIOKEY_13_Pre_AAAGATGAGCAGGCTA-1,1000,627,BIOKEY_13,Pre,NaN,HER2+,CD4_N,treatment_naive
3,BIOKEY_13_Pre_AAAGATGCAAGCCGTC-1,1288,681,BIOKEY_13,Pre,NaN,HER2+,CD8_EM,treatment_naive
4,BIOKEY_13_Pre_AAAGATGGTTTGACAC-1,2056,789,BIOKEY_13,Pre,NaN,HER2+,CD8_N,treatment_naive
...,...,...,...,...,...,...,...,...,...
53377,BIOKEY_24_On_TTGTAGGTCACTGGGC-1,2639,981,BIOKEY_24,On,NE,ER+,CD8_EM,treatment_naive
53378,BIOKEY_24_On_TTTACTGCACCATCCT-1,2409,995,BIOKEY_24,On,NE,ER+,CD8_EM,treatment_naive
53379,BIOKEY_24_On_TTTACTGTCCATGCTC-1,1915,893,BIOKEY_24,On,NE,ER+,CD4_EM,treatment_naive
53380,BIOKEY_24_On_TTTGCGCCACTGTTAG-1,2602,1076,BIOKEY_24,On,NE,ER+,CD8_EM,treatment_naive


In [9]:
import pandas as pd
from scipy.io import mmread
from scipy.sparse import csr_matrix
from anndata import AnnData

# Define the directory containing your data
data_path = '/Genomics/pritykinlab/dillon/other_studies/BassezLambrechts2021/10x_Tcell'

# Paths to the files
matrix_path = f'{data_path}/matrix.mtx'
genes_path = f'{data_path}/genes.tsv'
barcodes_path = f'{data_path}/barcodes.tsv'

# Load the matrix
expr_matrix = mmread(matrix_path).tocsr().T

# Load genes and barcodes
genes = pd.read_csv(genes_path, header=None, sep='\t')[0].tolist()
barcodes = pd.read_csv(barcodes_path, header=None, sep='\t')[0].tolist()


In [10]:
# Create AnnData object
adata = AnnData(X=expr_matrix)

/tmp/ipykernel_1319338/1710061141.py:2: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = AnnData(X=expr_matrix)


In [11]:
adata.var_names = list(genes)
adata.obs_names = list(barcodes)

In [13]:
adata

AnnData object with n_obs × n_vars = 53382 × 25288